In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("check_kalman_filter.ipynb")

# Kalman filter 

This Jupyter notebook file imports (and tests) all of the code needed to do the Kalman' filter assignment. Note that the actual code is in the .py files in this directory.

This is where you'll fill in the missing code from the last assignment

Slides for this assignment: https://docs.google.com/presentation/d/1Q6w-vczvWHanGDqbuz6H1qhTOkSrX54kf1g8NgTcipQ/edit?usp=sharing

## Robot ground truth
This gets all the code in **robot_ground_truth.py**. If you change any code in that file then you will need to re-execute this import.

In [2]:
# These commands will force JN to actually re-load the external file when you re-execute the import command
%load_ext autoreload
%autoreload 2

In [3]:
from robot_ground_truth import RobotGroundTruth, test_continuous_move_functions

In [4]:
# Syntax check
robot_gt = RobotGroundTruth()
robot_gt.reset_location()
robot_gt.set_move_continuos_probabilities(sigma=0.01)
ret_amt_moved = robot_gt.move_continuous(amount=0.0)
if -0.1 < ret_amt_moved < 0.1:
    print(f"Robot ground truth: Passed move continuous syntax check")

Robot ground truth: Passed move continuous syntax check


In [5]:
# Edit the code in robot_ground_truth.py (the methods tagged with Kalman assignment)
# This checks that you are correctly generating the move continuous function in robot ground truth
#    If you call robot_gt.move_continous(amount), you should get back amount += some noise, and robot_gt.robot_location should change by amount +- some noise
#    If this fails, use the above cell to print out a few rounds of calling move continuous with different amounts and check that it is adding a reasonable amount of noise
test_continuous_move_functions()

Checking move with normal distribution probabilities
Passed continuous tests


True

In [6]:
grader.check("robotGT")

robotGT results: All test cases passed!

## Robot sensors

In [7]:
from robot_sensors import RobotSensors, test_continuous_sensor

In [8]:
# Syntax check
robot_gt = RobotGroundTruth()
robot_sensor = RobotSensors()
robot_sensor.set_distance_wall_sensor_probabilities(sigma=0.01)
dist_with_noise = robot_sensor.query_distance_to_wall(robot_gt)
if 0.0 <= dist_with_noise <= 1.0:
    print("Dist wall sensor, passed syntax test")

Dist wall sensor, passed syntax test


In [9]:
# Edit the code in robot_sensors.py (the methods tagged with Bayes assignment)
#  The sensor should be returning the robot_gt.robot_location +- some noise
#  Again, print out the values you're getting in the previous cell to see if they make sense if this test is failing
test_continuous_sensor()

Checking query wall with normal distribution probabilities
Passed continuous tests


True

In [10]:
grader.check("robot_sensors")

robot_sensors results: All test cases passed!

## Kalman filter

This is the part where you actually *do* the kalman filter update. There's two parts, the sensor update and the move update. The test file runs a sensor, then a move, then a sequence of sensor, sequence of move, then a mix.

The first cell just checks that you are calling all the functions correctly. You are free to edit it to print out the values at each step so you can debug each step in turn. If the entire test is failing, put in values into the methods and check that they "make sense"

In [13]:
from kalman_filter import KalmanFilter
from make_tests import test_kalman_update

In [71]:
# Syntax checks
kalman_filter = KalmanFilter()
robot_ground_truth = RobotGroundTruth()
robot_sensor = RobotSensors()

# Set mu/sigmas for sensor and movement
sensor_noise = {"mu": 0.0, "sigma": 0.1}
move_error = {"mu": 0.0, "sigma": 0.05}
robot_ground_truth.set_move_continuos_probabilities(move_error["sigma"])     # This is the R of the action (Ax + Bu + R)...
robot_sensor.set_distance_wall_sensor_probabilities(sensor_noise["sigma"])   # This is also your Q value in the Kalman update...

print(f"Robot location {robot_ground_truth.robot_loc}")
# Actually get out a distance reading from the robot sensor. Should return 0.5 +- some noise
dist_reading = robot_sensor.query_distance_to_wall(robot_gt=robot_ground_truth)
print(f"Distance reading from sensor: {dist_reading}")
# Actually move the robot
robot_ground_truth.move_continuous(amount=-0.1)
# Should be 0.4 +- some noise
print(f"New robot location after move {robot_ground_truth.robot_loc}")

# The two kalman filter update methods. You can put in whatever values you want to test
kalman_filter.update_belief_distance_sensor(robot_sensor, dist_reading=dist_reading)
kalman_filter.update_continuous_move(robot_ground_truth, amount=-0.1)

# Example print statement 
print(f"Location Gaussian is: {kalman_filter.location_mean()}, sigma is {kalman_filter.location_sigma()}")

Robot location 0.5
Distance reading from sensor: 0.4810928333564287
New robot location after move 0.37919009924349256
Location Gaussian is: 0.38220501962958, sigma is 0.10914103126634987


In [15]:
# GUIDE If this is failing, debug each step (Ax + Bu + noise) and (x = Cz) in turn by putting values in the calls in the previous cell
from make_tests import test_kalman_update
test_kalman_update()

Testing Kalman


ValueError: Failed sequence ['Sensor'], got mu 0.668341503381677, expected 0.6430902778744254

In [ ]:
grader.check("kalman_filter")

kalman_filter results:
    kalman_filter - 1 result:
        ❌ Test case failed
        Trying:
            assert test_kalman_update(b_print=False)
        Expecting nothing
        **********************************************************************
        Line 1, in kalman_filter 0
        Failed example:
            assert test_kalman_update(b_print=False)
        Exception raised:
            Traceback (most recent call last):
              File "/opt/homebrew/Cellar/python@3.13/3.13.7/Frameworks/Python.framework/Versions/3.13/lib/python3.13/doctest.py", line 1398, in __run
                exec(compile(example.source, filename, "single",
                ~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
                             compileflags, True), test.globs)
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
              File "<doctest kalman_filter 0[0]>", line 1, in <module>
                assert test_kalman_update(b_print=False)
                       ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^
              File "/Users/elliotzimmer/Documents/maker/Prog/VScodium/ROB456/RobotHallway/make_tests.py", line 154, in test_kalman_update
                raise ValueError(f"Failed sequence {seq['seq']}, got mu {kalman_filter.location_mean()}, expected {seq['mu']}")
            ValueError: Failed sequence ['Sensor'], got mu 0.5430902778744254, expected 0.6430902778744254

## Hours and collaborators
Required for every assignment - fill out before you hand-in.

Listing names and websites helps you to document who you worked with and what internet help you received in the case of any plagiarism issues. You should list names of anyone (in class or not) who has substantially helped you with an assignment - or anyone you have *helped*. You do not need to list TAs.

Listing hours helps us track if the assignments are too long.

In [ ]:

# List of names (creates a set)
worked_with_names = {"not filled out"}
# List of URLS W26 (creates a set)
websites = {"not filled out"}
# Approximate number of hours, including lab/in-class time
hours = -1.5

In [ ]:
grader.check("hours_collaborators")

### To submit

- Do a restart then run all to make sure everything runs ok
- Save the file
- Submit this .ipynb file and **robot_sensors.py** and **robot_ground_truth.py** and **kalman_filter.py** through gradescope, Homework Kalman filter
- You do NOT need to submit the data files or  **world_ground_truth.py** or **make_tests.py**
- Take out/suppress all print statements

If the Gradescope autograder fails, please check here first for common reasons for it to fail
    https://docs.google.com/presentation/d/1tYa5oycUiG4YhXUq5vHvPOpWJ4k_xUPp2rUNIL7Q9RI/edit?usp=sharing

Most likely failure for this assignment is not naming the data directory and files correctly; capitalization matters for the Gradescope grader. 

Second reason for failure is you forgot to include the **.py** files and/or you put the file(s) in a folder
